# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [2]:
#more imports
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('processed_data', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('processed_data', engine)
Y = df.iloc[:,4:]
X = df['message']
#category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # normalize, lemmatize and clean
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split data into train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# fit training data
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# predict
y_pred = pipeline.predict(X_test)

In [40]:
# Evaluate model
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy : {}'.format(accuracy))

def evaluate_perf(y_test, y_pred):
    for i, column in enumerate(y_test):
        print(column)
        print('\n')
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
        
evaluate_perf(y_test, y_pred)

accuracy : related                   0.792996
request                   0.885691
offer                     0.995608
aid_related               0.751618
medical_help              0.923139
medical_products          0.950185
search_and_rescue         0.973994
security                  0.981276
military                  0.969718
child_alone               1.000000
water                     0.951919
food                      0.923948
shelter                   0.921983
clothing                  0.985899
money                     0.978618
missing_people            0.988095
refugees                  0.965904
death                     0.957466
other_aid                 0.869279
infrastructure_related    0.934350
transport                 0.954924
buildings                 0.952843
electricity               0.978964
tools                     0.993874
hospitals                 0.990176
shops                     0.996070
aid_centers               0.986708
other_infrastructure      0.955617
weather_r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'clf__estimator__n_estimators': [5, 10, 20],
    'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3)

# fit model
cv.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, score=0.17228421771805966, total=  17.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.9s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, score=0.16943748576634024, total=  17.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   44.2s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.21840127533591436, total=  26.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.2246640856296971, total=  26.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.22250056934639034, total=  43.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.22455021635162833, total=  43.6s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5, score=0.18469596902755636, total=  16.1s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5 
[CV]  clf__estimator__min_samples_split=4, c

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  6.7min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [5, 10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [ ]:
#pipeline.get_params().keys()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
# Print best parameter
print(cv.best_params_)

# predict to test using best params
y_pred = cv.predict(X_test)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}


In [12]:
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy : {}'.format(accuracy))
print('\n')
evaluate_perf(y_test, y_pred)

accuracy : related                   0.792996
request                   0.885691
offer                     0.995608
aid_related               0.751618
medical_help              0.923139
medical_products          0.950185
search_and_rescue         0.973994
security                  0.981276
military                  0.969718
child_alone               1.000000
water                     0.951919
food                      0.923948
shelter                   0.921983
clothing                  0.985899
money                     0.978618
missing_people            0.988095
refugees                  0.965904
death                     0.957466
other_aid                 0.869279
infrastructure_related    0.934350
transport                 0.954924
buildings                 0.952843
electricity               0.978964
tools                     0.993874
hospitals                 0.990176
shops                     0.996070
aid_centers               0.986708
other_infrastructure      0.955617
weather_r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
# The accurate looks good, no need to train more models

0.75161812297734631

### 9. Export your model as a pickle file

In [21]:
# Save model as pickle
filename = "clf_model.pickle"

# save model
pickle.dump(best_model, open(filename, "wb"))

# load model
#loaded_model = pickle.load(open(filename, "rb"))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [44]:
import sys
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
#more imports
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


def load_data(database_filepath):
    engine = create_engine('sqlite:///{}'.format(database_filepath))
    df = pd.read_sql_table(database_filepath, engine)
    Y = df.iloc[:,4:]
    X = df['message']
    category_names = Y.columns.tolist()
    return X, Y, category_names


def tokenize(text):
     # normalize, lemmatize and clean
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens


def build_model():
    
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split = 2, n_estimators = 20)))
])
    return pipeline


def evaluate_model(model, X_test, Y_test):
    # predict to test using best params
    y_pred = model.predict(X_test)
    for i, column in enumerate(Y_test):
        print(column)
        print('\n')
        print(classification_report(Y_test.iloc[:,i], y_pred[:,i]))
        
        pass
        

def save_model(model, model_filepath):
    # save model
    pickle.dump(model, open(model_filepath, "wb"))
    pass


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model... .....')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


TypeError: main() got an unexpected keyword argument 'database_filepath'